# Lesson 10: Algorithms -- Graph Search Algorithm
----
In this lesson, we will cover the following parts:
* 10.1: Lecture Note
* 10.2: Leetcode Training (Basic)
* 10.3: Leetcode Practice (Advanced)

## 10.1 Lecture Note -- Graph Search Algorithm

### 10.1.1 Graph

#### Graph Definition  
In simple terms, a graph contains two things:
1. A set of vertices V.
2. A set of edges E. Every edge in E connects two vertices in V.
![Graph](source/lesson11_GraphSearch_Graph.png)

Based on the direction of en edge, we could directed graph or undirected graph. On the above example, the graph on the left side is an undirected graph. Using notation, 
* for the undirected graph, we could have:
  * $G_{left} = (V_{left}, E_{left})$
  * $V_{left} = {a, b, c}$
  * $E_{left} = {(a, b), (b, c), (a, c)}$
* for the directed graph, we could have:
  * $G_{right} = (V_{right}, E_{right})$
  * $V_{right} = {a, b, c}$
  * $E_{right} = {(a, b), (b, c), (a, c), (c,b)}$
  
As you can see, although these two look almost the same, the second edge set contains ordered pair of vertices.

#### Graph Application
Basically, almost all real world problems can be modeled as graph problem since we could model an entity as a vertex and the relationship between entities as edges.
1. Web crawling.
2. Social networking.
3. Grabage collections.
4. ...

#### Graph Representation
Based on the defintion, the simplest way to represent a graph in Python on which you can run your algorithm is to use list to store all those vertices and pair of vertices as edges (ordered or unordered depending on which type of graph you want to represent right now).

The disadvantage of doing this is that it will be very inefficient to answer the following simple yet very common query:  
Given two vertices V1 and V2, is there an edge between them?  
The only way you can answer this question is to traversal all the possible edges inside your graph.

Alternatively, assuming we are dealing with undirected graph for simplicity, we could store all the possible pair of vertices in a matrix by using vertices' names as labels:

|   | a  | b  | c  |
| :-| :- | :- | :- |
| a | 0  | 1  | 1  |
| b | 1  | 0  | 0  |
| c | 1  | 1  | 0  |

This is called adjacency matrix (above is an example for the graph on the left in the definition section). For any given cell (u, v), if the value of this cell is 1, it means there exists an edge between vertex u and vertex v. There are two major disadvantages:
1. If graph is sparse, the matrix will contain a lot of cells with 0. The storage for these cells are wasted.
2. Adding a new vertex to the graph will be difficult.

```
A[a][b] := 1 means there is an edge between a & b. 
C[a][b] := sum(A[a][x] * A[x][b]) over x belongs to all vertices.  C[a][b] = A * A

A * A = A ^ 2 := 任意两个点之间长度为2的路径的条数
A * A * A = A ^ 3 := 任意两个点之间长度为3的路径的条数
```

The adjacency matrix looks at the graph from an eagle view. Alternatively, we could start with looking at a much smaller scale: a vertex. If we store all the neighbor vertices of a vertex as well as this vertex itself, we have another representation called adjacency list.  
a: {b, c}  
c: {a, b}  
b: {a, c}  
If you use hashtable for the vertex and its associated neighbors, hashset for just the neighbors of a specific vertex. You have the following advantages:
1. You do not store edges that do not exist.
2. Fast query for edge existence.
3. Easy to add new vertex or new edge.

#### Graph Exploratin
Many variations:
1. Given a source vertex s and a target vertex t, is there a path from s to t?
2. Given a source vertex s, find all the reachable vertices and edges from s.
3. Given a graph, traverse all the nodes inside it.
4. ......

All these variations has one common task: given a vertex, how to traverse all the reachable vertices from it systematically?

Basically, the algorithm is easy to come up with: assumping our graph is represented by the adjacency list, we first traverse all the neighbors of vertex s, then we traverse all the neighbors of those vertices we have just traversed, and so on. One important thing here is to avoid duplication carefully. Otherwise, if the graph has just one cycle, your traversal algorithm will not be terminated.

This is what we called breadth first search algorithm since you will finish visiting all vertices that are reachable from source by k moves before visiting vertices that are reachable from source by k+1 moves. The algorithm is sketched as follow (pseudo code):
```python
def BFS(graph, s):
    frontier = [s]
    has_seen = set(s)
    while frontier:
        next = []
        for u in frontier:
            for v in neighbors(u):
                if v not in has_seen:
                    next.append(v)
                    has_seen.add(v)
        frontier = next        

# Time Complexity: O(V+E)
# V: number of vertices
# E: number of edges
```


### 10.1.2 Graph Search -- BFS, Bidirectional BFS and A\*

#### Question 1: Given a binary tree, return the zigzag level order traversal of its nodes' values.

For example:
```
Given binary tree [3, 9, 20, null, null, 15, 7]
          3
         / \
        9   20
           /  \
          15   7
return its zigzag level order traversal as:
[
  [3]
  [20, 9]
  [15, 7]
]
```
Key observations:
1. The order of traversing each level of nodes alternates.

In [17]:
from helper import TreeNode, BinaryTree

In [5]:
# BFS -- list

class Solution(object):
    def zigzag_level_order(self, root):
        """
        :type root: TreeNode
        :rtype: List[List[int]]
        """
        if not root:
            return []
        
        frontier = []
        frontier.append(root)
        
        reverse = False
        
        results = []
        
        while frontier:
            solution = []
            post = []
            for node in frontier:
                solution.append(node.val)
                if node.left:
                    post.append(node.left)
                if node.right:
                    post.append(node.right)
            frontier = post
            results.append(solution[::-1] if reverse else solution[:])
            reverse = not reverse
            
        return results
    
if __name__ == "__main__":
    root = TreeNode(3)
    root.left = TreeNode(9)
    root.right = TreeNode(20)
    root.right.left = TreeNode(15)
    root.right.right = TreeNode(7)

    soln = Solution()
    print(soln.zigzag_level_order(root))

[[3], [20, 9], [15, 7]]


In [6]:
# BFS deque

from collections import deque

class Solution(object):
    def zigzag_level_order(self, root):
        """
        :type root: TreeNode
        :rtype: List[List[int]]
        """
        if not root:
            return []
        
        frontier = deque()
        frontier.append((root, 0))
        
        reverse = False
        
        results = []
        solution = []
        
        prev_count = 0
        
        while frontier:
            node, count = frontier.popleft()
            if count != prev_count:                
                results.append(solution[::-1] if reverse else solution[:])
                reverse = not reverse
                solution = []
                prev_count = count
                        
            solution.append(node.val)
            if node.left:
                frontier.append((node.left, count + 1))
            if node.right:
                frontier.append((node.right, count + 1))            
        
        results.append(solution[::-1] if reverse else solution[:])
        
        return results
    
if __name__ == "__main__":
    root = TreeNode(3)
    root.left = TreeNode(9)
    root.right = TreeNode(20)
    root.right.left = TreeNode(15)
    root.right.right = TreeNode(7)

    soln = Solution()
    print(soln.zigzag_level_order(root))

[[3], [20, 9], [15, 7]]


#### Question 2: Shortest Cell Path
In a given grid of 0s and 1s, we have some starting row and column sr, sc and a target row and column tr, tc. Return the length of the shortest path from sr, sc to tr, tc that walks along 1 values only.

Each location in the path, including the start and the end, must be a 1. Each subsequent location in the path must be 4-directionally adjacent to the previous location.

It is guaranteed that grid[sr][sc] = grid[tr][tc] = 1, and the starting and target positions are different.

If the task is impossible, return -1.

Examples:
```
input:
grid = [[1, 1, 1, 1], [0, 0, 0, 1], [1, 1, 1, 1]]
sr = 0, sc = 0, tr = 2, tc = 0
output: 8
(The lines below represent this grid:)
1111
0001
1111

grid = [[1, 1, 1, 1], [0, 0, 0, 1], [1, 0, 1, 1]]
sr = 0, sc = 0, tr = 2, tc = 0
output: -1
(The lines below represent this grid:)
1111
0001
1011
```
Constraints:
* [time limit] 5000ms
* [input] array.array.integer grid
* 1 ≤ arr.length = arr[i].length ≤ 10
* [input] integer sr
* [input] integer sc
* [input] integer tr
* [input] integer tc
* All sr, sc, tr, tc are valid locations in the grid, grid[sr][sc] = grid[tr][tc] = 1, and (sr, sc) != (tr, tc).
* [output] integer

In [9]:
# BFS

class Solution(object):
    def shortestCellPath(self, grid, sr, sc, tr, tc):
        """
        @param grid: int[][]
        @param sr: int
        @param sc: int
        @param tr: int
        @param tc: int
        @return: int
        """
        frontier = [(sr, sc, 0)]
        visited = set()
        dr = [-1, 0, 1, 0]
        dc = [0, -1, 0, 1]

        while frontier:
            post = []
            for row, col, step in frontier:
                if row == tr and col == tc:
                    return step
                for d in range(4):
                    new_row = row + dr[d]
                    new_col = col + dc[d]
                    if (0 <= new_row < len(grid) and 0 <= new_col < len(grid[0]) and
                        grid[new_row][new_col] == 1 and (new_row, new_col) not in visited):
                        post.append((new_row, new_col, step + 1))
                        visited.add((new_row, new_col))
            frontier = post

        return -1 

if __name__ == "__main__":
    soln = Solution()    
    
    grid = [[1, 1, 1, 1], [0, 0, 0, 1], [1, 1, 1, 1]]
    sr, sc, tr, tc = 0, 0, 2, 0
    print(soln.shortestCellPath(grid, sr, sc, tr, tc))

    grid = [[1, 1, 1, 1], [0, 0, 0, 1], [1, 0, 1, 1]]
    sr, sc, tr, tc = 0, 0, 2, 0
    print(soln.shortestCellPath(grid, sr, sc, tr, tc))


    grid = [[1,1,0,0,1,0],[1,0,0,1,1,1],[0,0,0,0,1,1],[1,0,0,0,0,0],[0,1,1,0,0,1],[0,0,1,1,0,0]]
    sr, sc, tr, tc = 3, 0, 1, 3
    print(soln.shortestCellPath(grid, sr, sc, tr, tc))

8
-1
-1


In [10]:
# Bidirectional BFS

class Solution(object):
    def shortestCellPath(self, grid, sr, sc, tr, tc):
        """
        @param grid: int[][]
        @param sr: int
        @param sc: int
        @param tr: int
        @param tc: int
        @return: int
        """
        set1 = set()
        set1.add((sr, sc))
        set2 = set()
        set2.add((tr, tc))

        visited = set()
        visited.add((sr, sc))

        dx = [-1, 0, 1, 0]
        dy = [0, -1, 0, 1]

        count = 0

        while set1 and set2:
            if len(set1) > len(set2):
                set1, set2 = set2, set1

            post = []
            for x, y in set1:
                for d in range(4):
                    nx, ny = x + dx[d], y + dy[d]
                    if (nx, ny) in set2:
                        return count + 1
                    if (0 <= nx < len(grid) and 0 <= ny < len(grid[0]) and
                        grid[nx][ny] == 1 and (nx, ny) not in visited):
                        post.append((nx, ny))
                        visited.add((nx, ny))
            count += 1
            set1 = post

        return -1 

if __name__ == "__main__":
    soln = Solution()    
    
    grid = [[1, 1, 1, 1], [0, 0, 0, 1], [1, 1, 1, 1]]
    sr, sc, tr, tc = 0, 0, 2, 0
    print(soln.shortestCellPath(grid, sr, sc, tr, tc))

    grid = [[1, 1, 1, 1], [0, 0, 0, 1], [1, 0, 1, 1]]
    sr, sc, tr, tc = 0, 0, 2, 0
    print(soln.shortestCellPath(grid, sr, sc, tr, tc))


    grid = [[1,1,0,0,1,0],[1,0,0,1,1,1],[0,0,0,0,1,1],[1,0,0,0,0,0],[0,1,1,0,0,1],[0,0,1,1,0,0]]
    sr, sc, tr, tc = 3, 0, 1, 3
    print(soln.shortestCellPath(grid, sr, sc, tr, tc))

8
-1
-1


In [11]:
# A*
import heapq

class Solution(object):
    def shortestCellPath(self, grid, sr, sc, tr, tc):
        """
        @param grid: int[][]
        @param sr: int
        @param sc: int
        @param tr: int
        @param tc: int
        @return: int
        """
        # f = g + h
        heap = []
        # (f, g, x, y)
        heap.append((0, 0, sr, sc))
        costs = {}
        costs[(sr, sc)] = 0

        dx, dy = [-1, 0, 1, 0], [0, -1, 0, 1]

        while heap:
            f, g, x, y = heapq.heappop(heap)
            if (x, y) == (tr, tc):
                return g

            for d in range(4):
                nx, ny = x + dx[d], y + dy[d]

                if (0 <= nx < len(grid) and 0 <= ny < len(grid[0]) and 
                    grid[nx][ny] == 1):
                    ng = g + 1
                    nh = abs(nx - tr) + abs(ny - tc)
                    nf = ng + nh
                    if ((nx, ny) not in costs or costs[(nx, ny)] > nf):
                        costs[(nx, ny)] = nf
                        heapq.heappush(heap, (nf, ng, nx, ny))

        return -1 

if __name__ == "__main__":
    soln = Solution()    
    
    grid = [[1, 1, 1, 1], [0, 0, 0, 1], [1, 1, 1, 1]]
    sr, sc, tr, tc = 0, 0, 2, 0
    print(soln.shortestCellPath(grid, sr, sc, tr, tc))

    grid = [[1, 1, 1, 1], [0, 0, 0, 1], [1, 0, 1, 1]]
    sr, sc, tr, tc = 0, 0, 2, 0
    print(soln.shortestCellPath(grid, sr, sc, tr, tc))


    grid = [[1,1,0,0,1,0],[1,0,0,1,1,1],[0,0,0,0,1,1],[1,0,0,0,0,0],[0,1,1,0,0,1],[0,0,1,1,0,0]]
    sr, sc, tr, tc = 3, 0, 1, 3
    print(soln.shortestCellPath(grid, sr, sc, tr, tc))

8
-1
-1


#### Question 3: [Leetcode 200 Medium] [Number of Islands](https://leetcode.com/problems/number-of-islands/)

Given a 2d grid map of '1's (land) and '0's (water), count the number of islands. An island is surrounded by water and is formed by connecting adjacent lands horizontally or vertically. You may assume all four edges of the grid are all surrounded by water.

Example 1:
```
Input:
11110
11010
11000
00000

Output: 1
```

Example 2:
```
Input:
11000
11000
00100
00011

Output: 3
```

Analysis:  
Use BFS to find island (we can view each 1 as node, thus an island is equivalnet to a graph that consists of those nodes)

```python
has_seen = ...

for x in ...:
    for y in ...:
        if matrix[x][y] == 1 and not in has_seen:
            BFS(...)
            num_islands += 1

def BFS(matrix, x, y, has_seen):
    frontier = [(x, y)]
    while frontier:
        next = []
        for u in frontier:
            for v in neighbors(u):?
                (x-1, y), (x+1, y), (x, y-1), (x, y+1)
                if v not in has_seen:
                    next.append(v)
                    has_seen.add(v)
        frontier = next        
```

In [12]:
# BFS with a visited matrix

class Solution(object):
    def numIslands(self, grid):
        """
        :type grid: List[List[str]]
        :rtype: int
        """
        res, visited = 0, set()
        for row in range(0, len(grid)):
            for col in range(0, len(grid[0])):
                if grid[row][col] == 1 and (row, col) not in visited:
                    res += 1
                    self.BFS(grid, row, col, visited)
        return res
    
    def BFS(self, grid, row, col, visited):
        dr, dc = [-1, 0, 1, 0], [0, -1, 0, 1]
        visited.add((row, col))
        
        frontier = [(row, col)]
        while frontier:
            post = []
            for r, c in frontier:
                for d in range(4):
                    new_row, new_col = r + dr[d], c + dc[d]
                    if (0 <= new_row < len(grid) and 0 <= new_col < len(grid[0]) and
                       grid[new_row][new_col] == 1 and (new_row, new_col) not in visited):
                        node = (new_row, new_col)
                        visited.add(node)
                        post.append(node)
            frontier = post
            
if __name__ == "__main__":
    grid = [[1,1,1,1,0], 
        [1,1,0,1,0], 
        [1,1,0,0,0],
        [0,0,0,0,0]]
    soln = Solution()
    print(soln.numIslands(grid))

    grid = [[1,1,0,0,0], 
            [1,1,0,0,0], 
            [0,0,1,0,0],
            [0,0,0,1,1]]
    soln = Solution()
    print(soln.numIslands(grid))

    grid = [[1,1,0,1,1], 
            [1,1,0,0,1], 
            [0,0,1,0,0],
            [0,0,0,0,1]]
    soln = Solution()
    print(soln.numIslands(grid))

1
3
4


In [13]:
# BFS without a visited matrix

class Solution(object):
    def numIslands(self, grid):
        """
        :type grid: List[List[str]]
        :rtype: int
        """
        res = 0
        for row in range(0, len(grid)):
            for col in range(0, len(grid[0])):
                if grid[row][col] == 1:
                    res += 1
                    grid[row][col] = 0
                    self.BFS(grid, row, col)
        return res
    
    def BFS(self, grid, row, col):
        dr, dc = [-1, 0, 1, 0], [0, -1, 0, 1]
                
        frontier = [(row, col)]
        while frontier:
            post = []
            for r, c in frontier:
                for d in range(4):
                    new_row, new_col = r + dr[d], c + dc[d]
                    if (0 <= new_row < len(grid) and 0 <= new_col < len(grid[0]) and
                       grid[new_row][new_col] == 1):
                        grid[new_row][new_col] = 0
                        node = (new_row, new_col)
                        post.append(node)
            frontier = post
            
if __name__ == "__main__":
    grid = [[1,1,1,1,0], 
        [1,1,0,1,0], 
        [1,1,0,0,0],
        [0,0,0,0,0]]
    soln = Solution()
    print(soln.numIslands(grid))

    grid = [[1,1,0,0,0], 
            [1,1,0,0,0], 
            [0,0,1,0,0],
            [0,0,0,1,1]]
    soln = Solution()
    print(soln.numIslands(grid))

    grid = [[1,1,0,1,1], 
            [1,1,0,0,1], 
            [0,0,1,0,0],
            [0,0,0,0,1]]
    soln = Solution()
    print(soln.numIslands(grid))

1
3
4


In [14]:
# Backtracking

class Solution(object):
    def numIslands(self, grid):
        """
        :type grid: List[List[str]]
        :rtype: int
        """
        if not grid or not grid[0]:
            return 0
        
        res = 0
        for row in range(0, len(grid)):
            for col in range(0, len(grid[0])):
                if grid[row][col] == 1:
                    res += 1
                    grid[row][col] = 0
                    self.backtracking(grid, row, col)
        return res
    
    def backtracking(self, grid, row, col):
        # base case
        if row == len(grid) or col == len(grid[0]):
            return
        
        dr, dc = [-1, 0, 1, 0], [0, -1, 0, 1]

        for d in range(4):
            new_row, new_col = row + dr[d], col + dc[d]
            if (0 <= new_row < len(grid) and 0 <= new_col < len(grid[0]) and
               grid[new_row][new_col] == 1):
                grid[new_row][new_col] = 0
                self.backtracking(grid, new_row, new_col)
            
if __name__ == "__main__":
    grid = [[1,1,1,1,0], 
        [1,1,0,1,0], 
        [1,1,0,0,0],
        [0,0,0,0,0]]
    soln = Solution()
    print(soln.numIslands(grid))

    grid = [[1,1,0,0,0], 
            [1,1,0,0,0], 
            [0,0,1,0,0],
            [0,0,0,1,1]]
    soln = Solution()
    print(soln.numIslands(grid))

    grid = [[1,1,0,1,1], 
            [1,1,0,0,1], 
            [0,0,1,0,0],
            [0,0,0,0,1]]
    soln = Solution()
    print(soln.numIslands(grid))

1
3
4


#### Question 4: [Leetcode 127 Medium] [Word Ladder](https://leetcode.com/problems/word-ladder/)

Given two words (beginWord and endWord), and a dictionary's word list, find the length of shortest transformation sequence from beginWord to endWord, such that:
1. Only one letter can be changed at a time.
2. Each transformed word must exist in the word list. Note that beginWord is not a transformed word.

Note:
* Return 0 if there is no such transformation sequence.
* All words have the same length.
* All words contain only lowercase alphabetic characters.
* You may assume no duplicates in the word list.
* You may assume beginWord and endWord are non-empty and are not the same.

Example 1:
```
Input:
beginWord = "hit",
endWord = "cog",
wordList = ["hot","dot","dog","lot","log","cog"]

Output: 5

Explanation: As one shortest transformation is "hit" -> "hot" -> "dot" -> "dog" -> "cog",
return its length 5.
```

Example 2:
```
Input:
beginWord = "hit"
endWord = "cog"
wordList = ["hot","dot","dog","lot","log"]

Output: 0

Explanation: The endWord "cog" is not in wordList, therefore no possible transformation.
```

In [4]:
# BFS

import string

class Solution(object):
    def ladderLength(self, beginWord, endWord, wordList):
        """
        :type beginWord: str
        :type endWord: str
        :type wordList: List[str]
        :rtype: int
        """
        wordList = set(wordList)
        if endWord not in wordList:
            return 0
        
        ans = 1
        frontier = [beginWord]
        visited = set(frontier)
        
        while frontier:
            post = []
            # how many nodes in the frontier
            for word in frontier:
                # how many edges from the node
                for index in range(0, len(word)):
                    for char in string.ascii_lowercase:
                        newWord = word[:index] + char + word[index+1:]
                       
                        if newWord == endWord:
                            return ans + 1
                        if newWord in wordList and newWord not in visited:
                            visited.add(newWord)
                            post.append(newWord) 
                            print(post)
            frontier = post
            ans += 1
        
        return 0
    
if __name__ == "__main__":
    soln = Solution()

    beginWord = "hit"
    endWord = "cog"
    wordList = ["hot","dot","dog","lot","log","cog"]
    print(soln.ladderLength(beginWord, endWord, wordList))

    beginWord = "hit"
    endWord = "cog"
    wordList = ["hot","dot","dog","lot","log"]
    print(soln.ladderLength(beginWord, endWord, wordList))

['hot']
['dot']
['dot', 'lot']
['dog']
['dog', 'log']
5
0


In [16]:
# Bidirectional BFS

import string

class Solution(object):
    def ladderLength(self, beginWord, endWord, wordList):
        """
        :type beginWord: str
        :type endWord: str
        :type wordList: List[str]
        :rtype: int
        """
        wordList = set(wordList)
        if endWord not in wordList:
            return 0
        
        ans = 1
        set1 = set()
        set1.add(beginWord)
        set2 = set()
        set2.add(endWord)
        
        visited = set()
        visited.add(beginWord)
        
        while set1 and set2:
            if len(set1) > len(set2):
                set1, set2 = set2, set1
                
            post = []
            # how many nodes in the frontier
            for word in set1:
                # how many edges from the node
                for index in range(0, len(word)):
                    for char in string.ascii_lowercase:
                        newWord = word[:index] + char + word[index+1:]
                        if newWord in set2:
                            return ans + 1
                        if newWord in wordList and newWord not in visited:
                            visited.add(newWord)
                            post.append(newWord)            
            set1 = post
            ans += 1
        
        return 0
    
if __name__ == "__main__":
    soln = Solution()

    beginWord = "hit"
    endWord = "cog"
    wordList = ["hot","dot","dog","lot","log","cog"]
    print(soln.ladderLength(beginWord, endWord, wordList))

    beginWord = "hit"
    endWord = "cog"
    wordList = ["hot","dot","dog","lot","log"]
    print(soln.ladderLength(beginWord, endWord, wordList))

5
0


#### Question 5: [Leetcode 286] [Walls and Gates](https://zhuhan0.blogspot.com/2017/07/leetcode-286-walls-and-gates.html)

You are given a m x n 2D grid initialized with these three possible values.
1. -1 -- A wall or an obstacle.
2. 0 -- A gate.
3. INF -- Infinity means an empty room. We use the value 2^31 - 1 = 2147483647 to represent INF as you may assume that the distance to a gate is less than 2147483647.

Fill each empty room with the distance to its nearest gate. If it is impossible to reach a gate, it should be filled with INF.

For example, 
```
given the 2D grid:

INF  -1  0  INF
INF INF INF  -1
INF  -1 INF  -1
  0  -1 INF INF
  
After running your function, the 2D grid should be:

3  -1   0   1
2   2   1  -1
1  -1   2  -1
0  -1   3   4  
```
Thought process:  
Iterate through the grid. Add gates' coordinates to queue. BFS from gates and expand through all rooms.

In [24]:
# BFS with deque

from collections import deque

class Solution(object):
    def walls_and_gates(self, rooms):
        """
        :type rooms: List[List[int]]
        :rtype: void. Do not return anything, modify rooms in-place instead.
        """
        queue = deque()
        queue_num, INF = 0, 2147483647
        dx, dy = [-1, 0, 1, 0], [0, -1, 0, 1]
        
        # search for gate and store its location in the queue
        for row in range(len(rooms)):
            for col in range(len(rooms[0])):
                if rooms[row][col] == 0:
                    queue.append((row, col, 0))
                    queue_num += 1

        while queue:
            x, y, distance = queue.popleft()
            for d in range(4):
                nx, ny = x + dx[d], y + dy[d]
                if 0 <= nx < len(rooms) and 0 <= ny < len(rooms[0]) and rooms[nx][ny] == INF:
                    rooms[nx][ny] = distance + 1
                    queue.append((nx, ny, distance + 1))
                
if __name__ == "__main__":
    INF = 2147483647
    rooms = [
        [INF, -1,  0,   INF],
        [INF, INF, INF,  -1],
        [INF, -1,  INF,  -1],
        [0,   -1,  INF, INF]   
    ]

    soln = Solution()
    soln.walls_and_gates(rooms)
    print(rooms)

[[3, -1, 0, 1], [2, 2, 1, -1], [1, -1, 2, -1], [0, -1, 3, 4]]


In [22]:
# BFS with list

class Solution(object):
    def walls_and_gates(self, rooms):
        """
        :type rooms: List[List[int]]
        :rtype: void. Do not return anything, modify rooms in-place instead.
        """
        if not rooms or not rooms[0]:
            return rooms
        
        frontier = []    
        # search for gate and store its location in frontier
        for row in range(len(rooms)):
            for col in range(len(rooms[0])):
                if rooms[row][col] == 0:
                    frontier.append((row, col))

        dx, dy = [-1, 0, 1, 0], [0, -1, 0, 1]
        INF = 2147483647
        distance = 1
        
        while frontier:
            post = []
            for x, y in frontier:
                for d in range(4):
                    nx, ny = x + dx[d], y + dy[d]
                    if 0 <= nx < len(rooms) and 0 <= ny < len(rooms[0]) and rooms[nx][ny] == INF:
                        rooms[nx][ny] = distance
                        post.append((nx, ny))
                        
            frontier = post
            distance += 1

                
if __name__ == "__main__":
    INF = 2147483647
    rooms = [
        [INF, -1,  0,   INF],
        [INF, INF, INF,  -1],
        [INF, -1,  INF,  -1],
        [0,   -1,  INF, INF]   
    ]

    soln = Solution()
    soln.walls_and_gates(rooms)
    print(rooms)

[[3, -1, 0, 1], [2, 2, 1, -1], [1, -1, 2, -1], [0, -1, 3, 4]]


#### Question 6: [Leetcode 864] [Shortest Path to Get All Keys](https://leetcode.com/problems/shortest-path-to-get-all-keys/)
Brain Teaser for next class:  
We are given a 2-dimensional grid. "." is an empty cell, "#" is a wall, "@" is the starting point, ("a", "b", ...) are keys, and ("A", "B", ...) are locks.

We start at the starting point, and one move consists of walking one space in one of the 4 cardinal directions.  We cannot walk outside the grid, or walk into a wall.  If we walk over a key, we pick it up.  We can't walk over a lock unless we have the corresponding key.

For some 1 <= K <= 6, there is exactly one lowercase and one uppercase letter of the first K letters of the English alphabet in the grid.  This means that there is exactly one key for each lock, and one lock for each key; and also that the letters used to represent the keys and locks were chosen in the same order as the English alphabet.

Return the lowest number of moves to acquire all keys.  If it's impossible, return -1. 

Example 1:
```
Input: ["@.a.#","###.#","b.A.B"]
Output: 8
```
Example 2:
```
Input: ["@..aA","..B#.","....b"]
Output: 6
```
Note:
1. 1 <= grid.length <= 30
2. 1 <= grid[0].length <= 30
3. grid[i][j] contains only '.', '#', '@', 'a'-'f' and 'A'-'F'
4. The number of keys is in [1, 6].  Each key has a different letter and opens exactly one lock.

In [23]:
from collections import deque

class Solution(object):
    def shortestPathAllKeys(self, grid):
        """
        :type grid: List[str]
        :rtype: int
        """
        if not grid or not grid[0]:
            return -1
        
        n, m = len(grid), len(grid[0])
        
        queue = deque()
        visited = set()
        num_keys = 0
        
        dx, dy = [-1, 0, 1, 0], [0, -1, 0, 1]
        
        # find the starting point and the number of keys
        for i in range(n):
            for j in range(m):
                if grid[i][j] == '@':
                    start = (i, j)
                elif grid[i][j] in "abcdef":
                    num_keys += 1
                    
        queue.append((start[0], start[1], 0, ".@abcdef", 0))
        
        while queue:
            x, y, steps, keys, collected_keys = queue.popleft()
            if grid[x][y] in "abcdef" and grid[x][y].upper() not in keys:
                keys += grid[x][y].upper()
                collected_keys += 1
            if collected_keys == num_keys:
                return steps
            
            for d in range(4):
                nx, ny = x + dx[d], y + dy[d]
                if (0 <= nx < n and 0 <= ny < m and 
                    grid[nx][ny] in keys and (nx, ny, keys) not in visited):
                    visited.add((nx, ny, keys))
                    queue.append([nx, ny, steps+1, keys, collected_keys])
                    
        return -1

if __name__ == "__main__":
    soln = Solution()
    print(soln.shortestPathAllKeys(grid=["@.a.#","###.#","b.A.B"]))

    print(soln.shortestPathAllKeys(grid=["@..aA","..B#.","....b"]))  

8
6


### 10.1.3 Graph Search Algorithm -- Depth First Search

#### Adjacency list graph representation - recite
![Graph](source/lesson11_GraphSearch_Graph.png)
Adjancency list stores the mapping between each vertex and its corresponding neighbor vertices. In Python, we usually represent this concept by a dictionary of which the key is the label of a vertex and the value is a list of labels of those neighbor vertices of this key. If we want to answer the following query quickly: given two vertices, do we have an edge between them? Then we could also use a set instead of a list as the value type of that dictionary.

With the above example, the adjacency list will look like the following:
```python
{
    'a': {'b', 'c'},
    'b': {'c'},
    'c': {'b'}
}
```

#### Depth First Search Algorithm
Assuming you are at the center of a maze, how will you find a way out? Basically, you will start with one direction, walk a little bit to the next junction, and then to decide which way you should go next and so on. If you have a very good memory, you will remember every decision you make so that when you hit a dead end, you can go back to the last junction and then make a new decision to try a different route. What kind of process that we have learnt in the previous lectures does it have the same structure as the one we have just described?

Yes. It is essentially backtracking during recursion, right? So, with recursion, we could very easily have the following graph search algorithm:  
Recursively explore the graph, backtracking as necessary.

Recipe:  
```python
def DFS(graph, visited, s):
    visit_status[s] = visiting
    # Recursively visit every reachable vertices from s that are still not being visited.
    for u in graph.neighbors(s):
        if u not in visited:
            visited.add(u)
            DFS(graph, visited, u)
    visit_status[s] = is_visited

def DFS_All(graph):
    visited = set()
    for v in graph:
        if v not in visited:
            visited.add(v)
            DFS(graph, visited, v)
```
Time Complexity:  
The total time complexity is the same as what we will have for BFS: $O(V+E)$. V means the total number of vertices, while E represents the total number of edges.

*Unique Properties for DFS* 

DFS is great for edge classification. For a directed graph, you can have the following categories:
1. Tree edge: normal edge you will have during DFS process. The whole DFS will give you a DFS spanning tree of the input graph. DFS过程中走过的边/路径，实际上就是树。因为DFS不会走回头路。剩余的边（DFS没有走过的边）就可以划分为：
2. Forward edge: edge that will point from a node to its descendant in the DFS spanning tree.
3. Backward edge: edge that will point from a node to its ancestor in the DFS spanning tree.
4. Cross edge: between two nodes that reside in two non-ancestor related subtrees.

![Edge](source/lesson11_GraphSearch_Edge.png)

* All green edges are tree edges.
* All red edges are forward edges.
* All blue edges are backward edges.
* All purple edges are cross edges.

For undirected graph, what type of edges could you have?  
Ans: you could only have tree edges and backward edges.

Based on these properties, we immediately know how to detect cycles in a graph -- by examining whether this graph contains at least one back edge or not.

#### Question 1: [Leetcode 261] [Graph Valid Tree](https://zhuhan0.blogspot.com/2017/07/leetcode-261-graph-valid-tree.html)
Given n nodes labeled from 0 to n - 1 and a list of undirected edges (each edge is a pair of nodes), write a function to check whether these edges make up a valid tree.

Example 1:
```
Input: n = 5, and edges = [[0, 1], [0, 2], [0, 3], [1, 4]]
Output: True
```
Exmple 2:
```
Input: n = 5, and edges = [[0, 1], [1, 2], [2, 3], [1, 3], [1, 4]]
Output: False
```
Note: you can assume that no duplicate edges will appear in edges. Since all edges are undirected, [0, 1] is the same as [1, 0] and thus will not appear together in edges.

Key observations:  
1. Input is given as a list of edges. Based on our algorithm template, we need to first build an adjacency list.
2. Based on the four types of edges, we know that undirected graph only has tree edges or backward edges. If we mark every vertex during the current DFS visit with a special marker, then if we encounter a vertex that we have already marked, we know this is a backward edge since all the vertices we marked so far are ancestors of it.
3. One caveat is that, given edge (u,v), you will visit it twice. Thus, when you visit v, u has already been marked as visited. But this is not really a backward edge hence not really a cycle. Our implementation needs to be able to make a difference for this case.

In [26]:
# use defaultdict from collections

from collections import defaultdict

class Solution(object):
    def valid_tree(self, n, edges):
        """
        :type n: int
        :type edges: List[List[int]]
        :rtype: bool
        """
        visited = set()
        
        # build an adjacency list
        graph = defaultdict(list)
        for start, end in edges:
            graph[start].append(end)
            graph[end].append(start)
        #print(graph)
        
        if self.has_cycle(graph, visited, -1, 0):
            return False
        if len(visited) != n:
            return False
        
        return True
    
    def has_cycle(self, graph, visited, parent, u):
        visited.add(u)
        
        for v in graph[u]:
            if v != parent:
                if v in visited:
                    return True
                elif self.has_cycle(graph, visited, u, v):
                    return True
                
        return False

if __name__ == "__main__":
    soln = Solution()
    #print(soln.valid_tree(n=5, edges=[[0, 1], [0, 2], [0, 3], [1, 4]]))
    print(soln.valid_tree(n=5, edges=[[0, 1], [1, 2], [2, 3], [1, 3], [1, 4]]))
 

True
False


In [27]:
# use dictionary

class Solution(object):
    def valid_tree(self, n, edges):
        """
        :type n: int
        :type edges: List[List[int]]
        :rtype: bool
        """
        visited = set()
        
        # build an adjacency list
        graph = {}
        for start, end in edges:
            if start not in graph:
                graph[start] = []
            graph[start].append(end)
            
            if end not in graph:
                graph[end] = []
            graph[end].append(start)
        #print(graph)
        
        if self.has_cycle(graph, visited, -1, 0):
            return False
        if len(visited) != n:
            return False
        
        return True
    
    def has_cycle(self, graph, visited, parent, u):
        visited.add(u)
        
        if u in graph:
            for v in graph[u]:
                if v != parent:
                    if v in visited:
                        return True
                    elif self.has_cycle(graph, visited, u, v):
                        return True

            return False

if __name__ == "__main__":
    soln = Solution()
    print(soln.valid_tree(n=5, edges=[[0, 1], [0, 2], [0, 3], [1, 4]]))
    print(soln.valid_tree(n=5, edges=[[0, 1], [1, 2], [2, 3], [1, 3], [1, 4]]))
 

True
False


#### Question 2: How to determine whether the given directed graph has a cycle or not.

For every node, we use 3 states to represent its state during DFS exploration.
1. Not visited
2. Visiting
3. Visited

u-> v  
0 -> 1 -> 2 ... 0
1. v is not visited => edge uv is Tree Edge
2. v is visiting    => edge uv is Backward Edge
3. v is visited     => either cross or forward

Key Observations:
1. Since directed graph can have cross edge or forward edge, if we just mark a vertex as visited, we could not embed the ancestor-descendant information into it. For example: 
![cycle](source/lesson11_GraphSearch_Cycle.png)
visit_status: {0: 0, 1: 0, 2: 0}

In [28]:
# please debug the following code

class Solution(object):
    def valid_tree(self, n, edges):
        """
        :type n: int
        :type edges: List[List[int]]
        :rtype: bool
        """
        visited = {}
        
        # build an adjacency list
        graph = defaultdict(list)
        for start, end in edges:
            graph[start].append(end)
        print(graph)
        
        for node in graph:
            print(graph)
            if node not in visited and self.has_cycle(graph, visited, node):
                return True
        
        return False
    
    def has_cycle(self, graph, visited, u):
        visited[u] = 0  # visiting
        print(u)
        for v in graph[u]:            
            if v not in visited:
                if self.has_cycle(graph, visited, v):
                    return True
            elif visited[v] == 0:
                return True
        
        visited[u] = 1  # visited
                
        return False

if __name__ == "__main__":
    soln = Solution()
    print(soln.valid_tree(n=5, edges=[[0, 1], [1, 2], [2, 3], [3, 0]]))
    print(soln.valid_tree(n=5, edges=[[0, 1], [1, 2], [2, 3], [0, 3], [3, 4]]))
 

defaultdict(<class 'list'>, {0: [1], 1: [2], 2: [3], 3: [0]})
defaultdict(<class 'list'>, {0: [1], 1: [2], 2: [3], 3: [0]})
0
1
2
3
True
defaultdict(<class 'list'>, {0: [1, 3], 1: [2], 2: [3], 3: [4]})
defaultdict(<class 'list'>, {0: [1, 3], 1: [2], 2: [3], 3: [4]})
0
1
2
3
4


RuntimeError: dictionary changed size during iteration

In [3]:
class Solution(object):
    def valid_tree(self, n, edges):
        """
        :type n: int
        :type edges: List[List[int]]
        :rtype: bool
        """
        visited = {}
        
        # build an adjacency list
        graph = {}
        for start, end in edges:
            if start not in graph:
                graph[start] = []
            graph[start].append(end)
        print(graph)
        
        for node in graph:
            if node not in visited and self.has_cycle(graph, visited, node):
                return True
        
        return False
    
    def has_cycle(self, graph, visited, u):
        visited[u] = 0  # visiting

        if u in graph:
            for v in graph[u]:            
                if v not in visited:
                    if self.has_cycle(graph, visited, v):
                        return True
                elif visited[v] == 0:
                    return True
        
        visited[u] = 1  # visited
                
        return False

if __name__ == "__main__":
    soln = Solution()
    print(soln.valid_tree(n=5, edges=[[0, 1], [1, 2], [2, 3], [3, 0]]))
    print(soln.valid_tree(n=5, edges=[[0, 1], [1, 2], [2, 3], [0, 3], [3, 4]]))
 

{0: [1], 1: [2], 2: [3], 3: [0]}
True
{0: [1, 3], 1: [2], 2: [3], 3: [4]}
False


#### Question 3: [Leetcode 785] [Is Graph Bipartite?](https://leetcode.com/problems/is-graph-bipartite/)
Given an undirected graph, return true if and only if it is bipartite.双边的

Recall that a graph is bipartite if we can split it's set of nodes into two independent subsets A and B such that every edge in the graph has one node in A and another node in B.

The graph is given in the following form: graph[i] is a list of indexes j for which the edge between nodes i and j exists.  Each node is an integer between 0 and graph.length - 1.  There are no self edges or parallel edges: graph[i] does not contain i, and it doesn't contain any element twice.

Example 1:
```
Input: [[1,3], [0,2], [1,3], [0,2]]
Output: True
Explanation: 
The graph looks like this:
0----1
|    |
|    |
3----2
We can divide the vertices into two groups: {0, 2} and {1, 3}.
```
Example 2:
```
Input: [[1,2,3], [0,2], [0,1,3], [0,2]]
Output: False
Explanation: 
The graph looks like this:
0----1
| \  |
|  \ |
3----2
We cannot find a way to divide the set of nodes into two independent subsets.
``` 
Note:
* graph will have length in range [1, 100].
* graph[i] will contain integers in range [0, graph.length - 1].
* graph[i] will not contain i or duplicate values.
* The graph is undirected: if any element j is in graph[i], then i will be in graph[j].

Key Observations:  
1. This problem is the same as we try to color every vertex in this graph with two colors.
2. Try the undirected graph that only contains 3 nodes, what do you discover?
3. Try the undirected graph that contains only 4 nodes like the following: 0 --> 1 -- > 2 --> 3 --> 0, what do you discover?
4. Add another edge between 0 and 2 (or, 1 and 3), what do you discover?

In [30]:
# Solution 1: DFS

class Solution:
    def isBipartite(self, graph: 'List[List[int]]') -> 'bool':
        colors = {}
        
        graphs = {}
        for start, output_list in enumerate(graph):
            if start not in graphs:
                graphs[start] = []
            graphs[start] = output_list
        print(graphs)
        
        for node in graphs:
            if node not in colors and not self.can_color(graphs, colors, node, 1):
                return False
        return True
    
    def can_color(self, graphs, colors, u, color):
        colors[u] = color
        
        if u in graphs:
            for v in graphs[u]:
                if v not in colors:
                    if not self.can_color(graphs, colors, v, -color):
                        return False
                elif colors[v] == color:  # v has the same color as its parent u
                    return False
        
        return True
    
if __name__ == "__main__":
    soln = Solution()
    print(soln.isBipartite(graph=[[1,3], [0,2], [1,3], [0,2]]))
    print(soln.isBipartite(graph=[[1,2,3], [0,2], [0,1,3], [0,2]]))

{0: [1, 3], 1: [0, 2], 2: [1, 3], 3: [0, 2]}
True
{0: [1, 2, 3], 1: [0, 2], 2: [0, 1, 3], 3: [0, 2]}
False


In [32]:
# Solution 2: BFS
class Solution:
    def isBipartite(self, graph: 'List[List[int]]') -> 'bool':
        colors = {}
        
        graphs = {}
        for start, output_list in enumerate(graph):
            if start not in graphs:
                graphs[start] = []
            graphs[start] = output_list
        print(graphs)
        
        for node in graphs:
            if node not in colors and not self.can_color(graphs, colors, node):
                return False
        return True
    
    def can_color(self, graphs, colors, u):
        colors[u] = 0
        frontier = [u]
        while frontier:
            post = []
            for u in frontier:
                for v in graphs[u]:
                    if v not in colors:
                        colors[v] = colors[u] + 1
                        post.append(v)
                    elif colors[v] == colors[u]:  # v has the same color as its parent u
                        return False
            frontier = post
        return True
    
if __name__ == "__main__":
    soln = Solution()
    print(soln.isBipartite(graph=[[1,3], [0,2], [1,3], [0,2]]))
    print(soln.isBipartite(graph=[[1,2,3], [0,2], [0,1,3], [0,2]]))

{0: [1, 3], 1: [0, 2], 2: [1, 3], 3: [0, 2]}
True
{0: [1, 2, 3], 1: [0, 2], 2: [0, 1, 3], 3: [0, 2]}
False


#### Topological Sort


<img src="source/lesson11_GraphSearch_TopolocialSort1.png">

<img src="source/lesson11_GraphSearch_TopolocialSort2.png">

#### Question 4: [Leetcode 210 Medium] [Course Schedule II](https://leetcode.com/problems/course-schedule-ii/)

There are a total of n courses you have to take, labeled from 0 to n-1.

Some courses may have prerequisites, for example to take course 0 you have to first take course 1, which is expressed as a pair: [0,1]

Given the total number of courses and a list of prerequisite pairs, return the ordering of courses you should take to finish all courses.

There may be multiple correct orders, you just need to return one of them. If it is impossible to finish all courses, return an empty array.

Example 1:
```
Input: 2, [[1,0]] 
Output: [0,1]
Explanation: There are a total of 2 courses to take. To take course 1 you should have finished   
             course 0. So the correct course order is [0,1] .
```

Example 2:
```
Input: 4, [[1,0],[2,0],[3,1],[3,2]]
Output: [0,1,2,3] or [0,2,1,3]
Explanation: There are a total of 4 courses to take. To take course 3 you should have finished both     
             courses 1 and 2. Both courses 1 and 2 should be taken after you finished course 0. 
             So one correct course order is [0,1,2,3]. Another correct ordering is [0,2,1,3] .
```

Note:
1. The input prerequisites is a graph represented by a list of edges, not adjacency matrices. Read more about how a graph is represented.
2. You may assume that there are no duplicate edges in the input prerequisites.

In [8]:
class Solution(object):
    def findOrder(self, numCourses, prerequisites):
        """
        :type numCourses: int
        :type prerequisites: List[List[int]]
        :rtype: List[int]
        """
        graph = {}
        for end, start in prerequisites:
            if start not in graph:
                graph[start] = []
            graph[start].append(end)
        print(graph)
        
        courses = []
              
        
        # find the nodes with zero in-degree 
        frontier = []
        for node in graph:
            if self.check_indegree_zero(graph, node):
                frontier.append(node)
        
        
        self.bfs(frontier, graph, courses)
        
        return courses if len(courses) == numCourses else []
    
    def bfs(self, frontier, graph, courses):
        
        while frontier:
            post = []
            for u in frontier:
                courses.append(u)
                while u in graph and graph[u]:
                    v = graph[u].pop()
                    if self.check_indegree_zero(graph, v):
                        post.append(v)
            frontier = post
                                

    def check_indegree_zero(self, graph, node):
        for _, neighbors in graph.items():
            if node in neighbors:
                return False
            
        return True
    
if __name__ == "__main__":
    soln = Solution()
    print(soln.findOrder(numCourses=4, prerequisites=[[1,0],[2,0],[3,1],[3,2]]))
    print(soln.findOrder(numCourses=4, prerequisites=[[1,0],[2,0],[1,2],[2,1]]))

{0: [1, 2], 1: [3], 2: [3]}
[0, 2, 1, 3]
{0: [1, 2], 2: [1], 1: [2]}
[]


In [1]:
class Solution(object):
    def findOrder(self, numCourses, prerequisites):
        """
        :type numCourses: int
        :type prerequisites: List[List[int]]
        :rtype: List[int]
        """
        graph = {}
        for end, start in prerequisites:
            if start not in graph:
                graph[start] = []
            graph[start].append(end)
        print(graph)
        
        courses = []
        visited = {}
        
        for course in graph:
            if course not in visited and self.has_cycle(graph, courses, course, visited):
                return []
        
        return courses[::-1]
    
    def has_cycle(self, graph, courses, u, visited):
        visited[u] = 0  # visiting
        
        if u in graph:
            for v in graph[u]:
                if v not in visited:
                    if self.has_cycle(graph, courses, v, visited):
                        return True
                elif visited[v] == 0:
                    return True
                
        visited[u] = 1  # visited
        courses.append(u)
        
        return False
    
if __name__ == "__main__":
    soln = Solution()
    print(soln.findOrder(numCourses=4, prerequisites=[[1,0],[2,0],[3,1],[3,2]]))
    print(soln.findOrder(numCourses=4, prerequisites=[[1,0],[2,0],[1,2],[2,1]]))

{0: [1, 2], 1: [3], 2: [3]}
[0, 2, 1, 3]
{0: [1, 2], 2: [1], 1: [2]}
[]


## 10.2 More on BFS

### 10.2.0 Introduction

#### When to use BFS

图的遍历 Traversal in Graph
* 层级遍历 Level Order Traversal
* 由点及面 Connected Component （连通问题）
* 拓扑排序 Topological Sorting

最短路径 Shortest Path in Simple Graph
* 仅限简单图求最短路径，即，图中每条边长度都是1，且没有方向

### 10.2.1 BFS in Binary Tree

#### Question: [Lintcode 69] [Binary Tree Level Order Traversal](https://www.jiuzhang.com/solutions/binary-tree-level-order-traversal/#tag-highlight-lang-python)

Description

Given a binary tree, return the level order traversal of its nodes' values. (ie, from left to right, level by level).
* The first data is the root node, followed by the value of the left and right son nodes, and "#" indicates that there is no child node.
* The number of nodes does not exceed 20.

Example 1:
```
Input：{1,2,3}
Output：[[1],[2,3]]
Explanation：
  1
 / \
2   3
it will be serialized {1,2,3} level order traversal
```

Example 2:
```
Input：{1,#,2,3}
Output：[[1],[2],[3]]
Explanation：
1
 \
  2
 /
3
it will be serialized {1,#,2,3} level order traversal
```

Challenge
* Challenge 1: Using only 1 queue to implement it.
* Challenge 2: Use BFS algorithm to do it.

In [2]:
class Solution:
    def levelOrder(self, root):
        """
        Solution: BFS -- list
        Time Complexity: O(n)
        Space Complexity: O(m)  # m -- maximum #node in a layer
        """
        # write your code here
        if not root:
            return []
        
        frontier = []
        frontier.append(root)
        
        results = []
        
        while frontier:
            res = []
            post = []
            for node in frontier:
                res.append(node.val)
                if node.left:
                    post.append(node.left)
                if node.right:
                    post.append(node.right)
            frontier = post
            results.append(res)
            
        return results

        
if __name__ == "__main__":
    root = TreeNode(3)
    root.left = TreeNode(9)
    root.right = TreeNode(20)
    root.right.left = TreeNode(15)
    root.right.right = TreeNode(7)

    soln = Solution()
    print(soln.levelOrder(root))

[[3], [9, 20], [15, 7]]


In [6]:
from collections import deque

class Solution:
    def levelOrder(self, root):
        """
        Solution: BFS -- list
        Time Complexity: O(n)
        Space Complexity: O(m)  # m -- maximum #node in a layer
        """
        # write your code here
        if not root:
            return []
        
        queue = deque()
        prev_count = 1
        
        queue.append((root, prev_count))
        
        results = []
        res = []
        
        while queue:
            node, count = queue.popleft()
            if count != prev_count:
                results.append(res[:])
                res = []
                prev_count = count
            
            res.append(node.val)
            if node.left:
                queue.append((node.left, count + 1))
            if node.right:
                queue.append((node.right, count + 1))
                
        results.append(res[:])            
            
        return results

        
if __name__ == "__main__":
    root = TreeNode(3)
    root.left = TreeNode(9)
    root.right = TreeNode(20)
    root.right.left = TreeNode(15)
    root.right.right = TreeNode(7)

    soln = Solution()
    print(soln.levelOrder(root))

[[3], [9, 20], [15, 7]]


#### Question: [Lintcode 7] [Serialize and Deserialize Binary Tree](https://www.jiuzhang.com/solutions/binary-tree-serialization/#tag-highlight-lang-python)

Description

Design an algorithm and write code to serialize and deserialize a binary tree. Writing the tree to a file is called 'serialization' and reading back from the file to reconstruct the exact same binary tree is 'deserialization'.

Example 1:
```
Input：{3,9,20,#,#,15,7}
Output：{3,9,20,#,#,15,7}
Explanation：
Binary tree {3,9,20,#,#,15,7},  denote the following structure:
	  3
	 / \
	9  20
	  /  \
	 15   7
it will be serialized {3,9,20,#,#,15,7}
```

Example 2:
```
Input：{1,2,3}
Output：{1,2,3}
Explanation：
Binary tree {1,2,3},  denote the following structure:
   1
  / \
 2   3
it will be serialized {1,2,3}
Our data serialization use BFS traversal. This is just for when you got Wrong Answer and want to debug the input.
```

You can use other method to do serializaiton and deserialization.

In [35]:
class Solution:
    def serialize(self, root):
        """
        An object of TreeNode, denote the root of the binary tree.
        This method will be invoked first, you should design your own algorithm 
        to serialize a binary tree which denote by a root node to a string which
        can be easily deserialized by your own "deserialize" method later.
        Solution: BFS
        Time Complexity: O(n)
        Space Complexity: O()
        """
        if not root:
            return ""
        
        frontier = []
        frontier.append(root)
        
        results = []
        
        while frontier:
            post = []
            for node in frontier:
                if node is not None:
                    results.append(str(node.val))
                    post.append(node.left)
                    post.append(node.right)
                else:
                    results.append("#")
                    
            frontier = post
            
        # remove the tail of '#'
        while results[-1] == "#":
            results.pop()                
            
        return ",".join(results)
        
    def deserialize(self, data):
        """
        A string serialized by your serialize method.
        This method will be invoked second, the argument data is what exactly
        you serialized at method "serialize", that means the data is not given by
        system, it's given by your own serialize method. So the format of data is
        designed by yourself, and deserialize it here as you serialize it in 
        "serialize" method.
        """
        data = data.split(',')
        
        if not data:
            return None
        
        root = TreeNode(int(data[0]))
        queue = [root]
        is_left_child = True
        index = 0
        
        for val in data[1:]:
            if val != '#':
                node = TreeNode(int(val))
                if is_left_child:
                    queue[index].left = node
                else:
                    queue[index].right = node
                queue.append(node)
                
            if not is_left_child:
                index += 1
            is_left_child = not is_left_child
            
        return root
    
if __name__ == "__main__":
    root = TreeNode(3)
    root.left = TreeNode(9)
    root.right = TreeNode(20)
    root.right.left = TreeNode(15)
    root.right.right = TreeNode(7)

    soln = Solution()
    data = soln.serialize(root)
    print(data)
    print(data.split(','))
    
    head = soln.deserialize(data)
    bt = BinaryTree()
    print(bt.BFS_traversal(head))

3,9,20,#,#,15,7
['3', '9', '20', '#', '#', '15', '7']
[[3], [9, 20], [' ', ' ', 15, 7]]


In [36]:
class Solution:
    def serialize(self, root):
        """
        An object of TreeNode, denote the root of the binary tree.
        This method will be invoked first, you should design your own algorithm 
        to serialize a binary tree which denote by a root node to a string which
        can be easily deserialized by your own "deserialize" method later.
        Solution: BFS
        Time Complexity: O(n)
        Space Complexity: O()
        """
        if not root:
            return ""
        
        queue = []
        queue.append(root)
        
        index = 0
        
        while index < len(queue):
            if queue[index] is not None:
                queue.append(queue[index].left)
                queue.append(queue[index].right)
            index += 1
            
        # remove the tail of None
        while queue[-1] is None:
            queue.pop()
            
        results = []
        for node in queue:
            if node is not None:
                results.append(str(node.val))
            else:
                results.append("#")
            
        return ",".join(results)
        
    def deserialize(self, data):
        """
        A string serialized by your serialize method.
        This method will be invoked second, the argument data is what exactly
        you serialized at method "serialize", that means the data is not given by
        system, it's given by your own serialize method. So the format of data is
        designed by yourself, and deserialize it here as you serialize it in 
        "serialize" method.
        """
        data = data.split(',')
        
        if not data:
            return None
        
        root = TreeNode(int(data[0]))
        queue = [root]
        is_left_child = True
        index = 0
        
        for val in data[1:]:
            if val != '#':
                node = TreeNode(int(val))
                if is_left_child:
                    queue[index].left = node
                else:
                    queue[index].right = node
                queue.append(node)
                
            if not is_left_child:
                index += 1
            is_left_child = not is_left_child
            
        return root
    
if __name__ == "__main__":
    root = TreeNode(3)
    root.left = TreeNode(9)
    root.right = TreeNode(20)
    root.right.left = TreeNode(15)
    root.right.right = TreeNode(7)

    soln = Solution()
    data = soln.serialize(root)
    print(data)
    print(data.split(','))
    
    head = soln.deserialize(data)
    bt = BinaryTree()
    print(bt.BFS_traversal(head))

3,9,20,#,#,15,7
['3', '9', '20', '#', '#', '15', '7']
[[3], [9, 20], [' ', ' ', 15, 7]]


#### [Lintcode 70] [Binary Tree Level Order Traversal II](https://www.jiuzhang.com/solutions/binary-tree-level-order-traversal-ii/#tag-highlight-lang-python)

Given a binary tree, return the bottom-up level order traversal of its nodes' values. (ie, from left to right, level by level from leaf to root).

Example 1:
```
Input:
{1,2,3}
Output:
[[2,3],[1]]
Explanation:
    1
   / \
  2   3
it will be serialized {1,2,3}
level order traversal
```

Example 2:
```
Input:
{3,9,20,#,#,15,7}
Output:
[[15,7],[9,20],[3]]
Explanation:
    3
   / \
  9  20
    /  \
   15   7
it will be serialized {3,9,20,#,#,15,7}
level order traversal
```

In [38]:
class Solution:
    """
    @param root: A tree
    @return: buttom-up level order a list of lists of integer
    """
    def levelOrderBottom(self, root):
        if not root:
            return []
        
        frontier = []
        frontier.append(root)
        
        results = []
        
        while frontier:
            post = []
            res = []
            for node in frontier:
                res.append(node.val)
                if node.left:
                    post.append(node.left)
                if node.right:
                    post.append(node.right)
            frontier = post
            results.append(res)
            
        return results[::-1]
    
if __name__ == "__main__":
    root = TreeNode(3)
    root.left = TreeNode(9)
    root.right = TreeNode(20)
    root.right.left = TreeNode(15)
    root.right.right = TreeNode(7)

    soln = Solution()
    print(soln.levelOrderBottom(root))   

[[15, 7], [9, 20], [3]]


#### [Lintcode 71] [Binary Tree Zigzag Level Order Traversal](https://www.jiuzhang.com/solutions/binary-tree-zigzag-level-order-traversal/#tag-highlight-lang-python)

Given a binary tree, return the zigzag level order traversal of its nodes' values. (ie, from left to right, then right to left for the next level and alternate between).

Example 1:
```
Input:{1,2,3}
Output:[[1],[3,2]]
Explanation:
    1
   / \
  2   3
```

Example 2:
```
Input:{3,9,20,#,#,15,7}
Output:[[3],[20,9],[15,7]]
Explanation:
    3
   / \
  9  20
    /  \
   15   7
```

In [42]:
class Solution:
    """
    @param root: A Tree
    @return: A list of lists of integer include the zigzag level order traversal of its nodes' values.
    """
    def zigzagLevelOrder(self, root):
        if not root:
            return []
        
        frontier = []
        frontier.append(root)
        
        results = []
        is_reversed = False
        
        while frontier:
            post = []
            res = []
            for node in frontier:
                res.append(node.val)
                if node.left:
                    post.append(node.left)
                if node.right:
                    post.append(node.right)
            frontier = post
            if is_reversed:
                results.append(res[::-1])
            else:
                results.append(res)
            is_reversed = not is_reversed
            
        return results
    
if __name__ == "__main__":
    root = TreeNode(3)
    root.left = TreeNode(9)
    root.right = TreeNode(20)
    root.right.left = TreeNode(15)
    root.right.right = TreeNode(7)

    soln = Solution()
    print(soln.zigzagLevelOrder(root))   

[[3], [20, 9], [15, 7]]


### 10.2.2 BFS in Graph

#### What is the difference between BFS in Graph and BFS in Binary Tree?
There might exist a cycle in a graph.  
图中存在环。存在环意味着，同一个节点可能重复进入队列

#### [Lintcode 178] [Graph Valid Tree](https://www.jiuzhang.com/solutions/graph-valid-tree/#tag-highlight-lang-python)

Given n nodes labeled from 0 to n - 1 and a list of undirected edges (each edge is a pair of nodes), write a function to check whether these edges make up a valid tree. 

You can assume that no duplicate edges will appear in edges. Since all edges are undirected, [0, 1] is the same as [1, 0] and thus will not appear together in edges.

Example 1:
```
Input: n = 5 edges = [[0, 1], [0, 2], [0, 3], [1, 4]]
Output: true.
```

Example 2:
```
Input: n = 5 edges = [[0, 1], [1, 2], [2, 3], [1, 3], [1, 4]]
Output: false.
```

In [49]:
from collections import deque

class Solution:
    """
    @param n: An integer
    @param edges: a list of undirected edges
    @return: true if it's a valid tree, or false
    """
    def validTree(self, n, edges):
        # write your code here
        if n <= 0 or not edges:
            return True
        
        # is cycle?
        if len(edges) != n - 1:
            return False
        
        # create adjacency list
        graph = {}
        for start, end in edges:
            if start not in graph:
                graph[start] = set()
            graph[start].add(end)
            if end not in graph:
                graph[end] = set()
            graph[end].add(start)
        print(graph)
        
        # is connected?
        # from any point to BFS        
        queue = deque()
        queue.append((0, 0))  # (node, level)
        
        visited = {}
        visited[0] = 1 # visited
        
        while queue:
            parent, count = queue.popleft()
            if parent in graph:
                for child in graph[parent]:
                    if child not in visited:
                        visited[child] = 1
                        queue.append((child, count + 1))
                    
        return len(visited) == n
                        

if __name__ == "__main__":
    soln = Solution()
    
    print(soln.validTree(n=5, edges=[[0, 1], [0, 2], [1, 3], [0, 4]]))
    print(soln.validTree(n=5, edges=[[0, 1], [1, 2], [2, 3], [1, 3], [1, 4]]))
    print(soln.validTree(n=5, edges=[[0, 1], [2, 3], [2, 4], [3, 4]]))

{0: {1, 2, 4}, 1: {0, 3}, 2: {0}, 3: {1}, 4: {0}}
True
False
{0: {1}, 1: {0}, 2: {3, 4}, 3: {2, 4}, 4: {2, 3}}
False


#### [Lintcode 137] [Clone Graph](https://www.jiuzhang.com/solutions/clone-graph/#tag-highlight-lang-python)

Clone an undirected graph. Each node in the graph contains a label and a list of its neighbors. Nodes are labeled uniquely.

You need to return a deep copied graph, which has the same structure as the original graph, and any changes to the new graph will not have any effect on the original graph.

Example1
```
Input:
{1,2,4#2,1,4#4,1,2}
Output: 
{1,2,4#2,1,4#4,1,2}
Explanation:
1------2  
 \     |  
  \    |  
   \   |  
    \  |  
      4  
```

In [ ]:
# Definition for a undirected graph node
class UndirectedGraphNode:
    def __init__(self, x):
        self.label = x
        self.neighbors = []

class Solution:
    """
    @param: node: A undirected graph node
    @return: A undirected graph node
    """
    def cloneGraph(self, node):
        # write your code here

In [50]:
# Definition for a undirected graph node
class UndirectedGraphNode:
    def __init__(self, x):
        self.label = x
        self.neighbors = []

class Solution:
    """
    @param: node: A undirected graph node
    @return: A undirected graph node
    """
    def cloneGraph(self, node):
        """
        DFS with memeory
        """
        # write your code here
        cache = {}
        return self.helper(node, cache)
    
    def helper(self, node, cache):
        # base case
        if not node:
            return None
        
        # memory
        if node.label in cache:
            return cache[node.label]
        
        # what to do in the current state
        root = UndirectedGraphNode(node.label)
        cache[node.label] = root
        
        # what to get from your children
        for child in node.neighbors:
            child_root = self.helper(child, cache)
            root.neighbors.append(child_root)
            
        # what to return to your parent
        return root

### 10.2.2 BFS in Grid

## 10.4 Leetcode Training (Basic)

[Leetcode 0127 Medium] [Word Ladder](Leetcode_0127.ipynb) (GraphSearch)  
[Leetcode 0126 Hard] [Word Ladder II](Leetcode_0126.ipynb) (GraphSearch)

[Leetcode 0200 Medium] [Number of Islands](Leetcode_0200.ipynb) (GraphSearch)

[Leetcode 0207 Medium] [Course Schedule](Leetcode_0207.ipynb) (GraphSearch)    
[Leetcode 0210 Medium] [Course Schedule II](Leetcode_0210.ipynb) (GraphSearch)

[Leetcode 0261 Medium] [Graph Valid Tree](Leetcode_0261.ipynb) (GraphSearch)

[Leetcode 0269 Hard] [Alien Dictionary](Leetcode_0269.ipynb) (GraphSearch)

[Leetcode 0323 Medium] [Number of Connected Components in an Undirected Graph](Leetcode_0323.ipynb) (GraphSearch)

## 10.5 Leetcode Practice (Advanced)

[Leetcode 0675 Hard] [Cut Off Trees for Golf Event](Leetcode_0675.ipynb) (GraphSearch)

[Leetcode 0733 Easy] [Flood Fill](Leetcode_0733.ipynb) (GraphSearch)

[Leetcode 0773 Hard] [Sliding Puzzle 华容道](Leetcode_0773.ipynb) (GraphSearch)

[Leetcode 0864 Hard] [Shortest Path to Get All Keys](Leetcode_0864.ipynb) (GraphSearch)

